# Phenylketonuria predictions with the human metabolic map RECON2

<p>**Authors**: Thierry D.G.A Mondeel, Stefania Astrologo, Ewelina Weglarz-Tomczak & Hans V. Westerhoff <br/>
University of Amsterdam <br/>2017
</p>

## Aim of this notebook
In this notebook we will aim to reproduce the inborn error of metabolism phenylketonuria (PKU) on the human metabolic reconstruction. In patients with PKU the enzyme converting phenylalanine into tyrosine (PAH) is not functional. 

## Setup
We start by setting up the python environment and loading RECON2. This part may take 10 seconds or so. 

In [1]:
import cobra
from cobra.flux_analysis import pfba
import pandas as pd # for nice tables
pd.set_option('display.max_colwidth', -1)

import escher
escher_colors = [{'type': 'min', 'color': '#cccccc', 'size': 5},
                                       {'type': 'mean', 'color': '#458b74', 'size': 15},
                                       {'type': 'max', 'color': '#ff7f24', 'size': 25}]

M = cobra.io.load_json_model('models/recon_2_2_simple_medium.json')
model = M.copy() # this way we can edit model but leave M unaltered

## Preliminary network investigation
<span style="color:red">**Assignment (3 min):**</span>
Try to print in the cell below all the reactions (not just their names but their reaction equations) that phenylalanine engages in. If you don't remember how to code this, refer back to the previous notebook. 

**Hint** write a for loop over the reactions attribute of the phenylalanine metabolite...

<span style="color:red">**Assignment (5 min):**</span>Compare reaction r0399 and the PHETHPTOX2 reaction. What is their relationship? Are they encoded by the same gene or different genes? Check this.

To investigate PKU we will ask the model for growth before and after the mutation in the PHETHPTOX2 gene. 

We require that there is always a base level of growth (0.5, arbitrary) so that the model is forced to predict a flux distribution in which growth is occuring.

We also made a so-called escher map on which to visualize our results. This makes it a bit more intuitive to see what is going on.

### Question: Are the PHETHPTOX2 and r0399 reactions essential? 
If yes, why? If not, why not?

### Deletions
We will now block r0399 and see what happens. 

In [ ]:
model.reactions.r0399.upper_bound = 0; model.reactions.r0399.lower_bound = 0;

ess_rxns2 = model.essential_reactions()

print '{} reactions are essential for growth\n\n'.format(len(ess_rxns2))

fbaTable = pd.DataFrame( { 'ID' : [rxn.id for rxn in ess_rxns2],
                'Name' : [rxn.name for rxn in ess_rxns2],
                'Reaction' : [rxn.reaction for rxn in ess_rxns2]  } )
fbaTable[['ID','Name','Reaction']]

### Question: What changed and why?

These reactions are newly essential

for rxn in ess_rxns2:
    if rxn not in ess_rxns :
        print '{} \t {} \t {}'.format(rxn.id,rxn.name,rxn.reaction)

### Assignment
Knockout the gene encoding both the PHETHPTOX2 and r0399 reactions and see what happens if you optimize for growth. What do you expect? Is that what you see?

We already wrote the code that knocks out the gene for you. You should add the FBA simulation and print the flux through the biomass reaction

model = M.copy() # start fresh

print model.reactions.PHETHPTOX2.genes # list the gene

model.genes.g1573.knock_out() # knock it out

# check that it worked
model.reactions.PHETHPTOX2.bounds

## PKU and brain disorders
Take look at the escher map we draw centered around the phenylalanine to tyrosine reaction. 

Note: This is only a subset of the reactions in RECON2. There are many more side branches we could draw here. We tried to emulate here the classical textbook we of representing the pathway.

Can you see from the map how PKU might lead to brain development issues?

In [2]:
pathway = escher.Builder(map_name='escher_map_RECON2_AA_metabolism', reaction_scale=escher_colors, 
                         reaction_data={}, reaction_styles=['color', 'size', 'text','abs'])
pathway.save_html('./predictions/empty_map.html',overwrite=True,enable_editing=False)
pathway.display_in_notebook()

Exception: Could not find the map escher_map_RECON2_AA_metabolism in the cache or on the server

In the following we will make various alterations to the model and visualize te flux distribution it predicts. We will project the results from flux balance analysis onto the escher map and print in text the flux variability analysis results for a couple key reactions.

Remember that flux balance analysis gives us one flux distribution that achieves the optimal values for the objective reaction. In contrast, flux variability analysis gives us the minimal and maximal flux possible through the reaction while maintaining steady-state balance. 

### When the medium includes tyrosine and phenylalanine

In [ ]:
model.reactions.biomass_reaction.lower_bound = 0.5
model.reactions.EX_tyr_L_e.lower_bound = -1

rxnsOfInterest = [ model.reactions.biomass_reaction, model.reactions.EX_phe_L_e, model.reactions.EX_tyr_L_e, model.reactions.PHETHPTOX2, \
                  model.reactions.r0399 ]

In [ ]:
fbasol = cameo.pfba(model)
fvasol = cameo.flux_variability_analysis(model,reactions=rxnsOfInterest,fraction_of_optimum=0)
print fvasol.data_frame

pathway = escher.Builder(map_name='escher_map_RECON2_AA_metabolism', reaction_scale=escher_colors, 
                         reaction_data={reaction: flux for reaction, flux in fbasol.fluxes.items() }, reaction_styles=['color', 'size', 'text','abs'])
pathway.save_html('./predictions/PKU_default.html',overwrite=True,enable_editing=False)
pathway.display_in_notebook()

### Question
Do you get why the enzyme catalyzing conversion of phenylalanine to tyrosine is not active here? 

## Medium without tyrosine 


In [ ]:
model.reactions.EX_tyr_L_e.lower_bound = 0

fbasol = cameo.pfba(model)
fvasol = cameo.flux_variability_analysis(model,reactions=rxnsOfInterest,fraction_of_optimum=0)
print fvasol.data_frame

pathway = escher.Builder(map_name='escher_map_RECON2_AA_metabolism', reaction_scale=escher_colors, 
                         reaction_data={reaction: flux for reaction, flux in fbasol.fluxes.items() }, reaction_styles=['color', 'size', 'text','abs'])
pathway.save_html('./predictions/PKU_noTyrosine.html',overwrite=True)
pathway.display_in_notebook()

### Question 
* How is it possible that there is still equal biomass reaction flux?
* The flux variability result shows us the interval for EX_phe_L_e is entirely negative. What does this mean?
* Why is PHETHPTOX2 not essential here? I.e. why does FVA allow its minimu to be zero?

Note: The recycling pathway of biopterin.

## Can we knock  out one of the two conversion enzymes and get the other one?

In [ ]:
model.reactions.r0399.upper_bound = 0;

fbasol = cameo.pfba(model)
fvasol = cameo.flux_variability_analysis(model,reactions=rxnsOfInterest,fraction_of_optimum=0,remove_cycles=True)
print fvasol.data_frame

pathway = escher.Builder(map_name='escher_map_RECON2_AA_metabolism', reaction_scale=escher_colors, 
                         reaction_data={reaction: flux for reaction, flux in fbasol.fluxes.items() }, reaction_styles=['color', 'size', 'text','abs'])
pathway.save_html('./predictions/PKU_noTyrosine_nor0399.html',overwrite=True)
pathway.display_in_notebook()

### Question
Is PHETHPTOX2 now essential? ie. is it required to carry flux to get growth?

## knockout PAH and its alternative reaction by knocking out the gene

In [ ]:
model.reactions.PHETHPTOX2.genes

### ASSIGNMENT
In the cell below add the code to knockout the gene and see what happens. Does the result make sense?

In [ ]:
### KNOCKOUT THE GENE HERE

fbasol = cameo.pfba(model)
fvasol = cameo.flux_variability_analysis(model,reactions=rxnsOfInterest,fraction_of_optimum=0,remove_cycles=True)
print fvasol.data_frame

pathway = escher.Builder(map_name='escher_map_RECON2_AA_metabolism', reaction_scale=escher_colors, 
                         reaction_data={reaction: flux for reaction, flux in fbasol.fluxes.items() }, reaction_styles=['color', 'size', 'text','abs'])
pathway.save_html('./predictions/PKU_noTyrosine_nor0399_noPhethptox2.html',overwrite=True)
pathway.display_in_notebook()

## Can we restore life through food?
* before we were missing the backward step from 34hpp (hydroxy phenyl pyruvate): it was not present in the medium by assumption
* Allowing this metabolite in the medium restores growth capability since the metabolic map allows the side branch pathway to run in reverse
* The thermodynamics of this reaction are questionable

In [ ]:
model = M.copy()
model.reactions.EX_34hpp.lower_bound = -1; 
rxnsOfInterest = [ model.reactions.biomass_reaction, model.reactions.EX_phe_L_e, model.reactions.EX_tyr_L_e, model.reactions.PHETHPTOX2, \
                  model.reactions.r0399, model.reactions.EX_34hpp]

model.genes.g1573.knock_out()

fbasol = cameo.pfba(model)
fvasol = cameo.flux_variability_analysis(model,reactions=rxnsOfInterest,fraction_of_optimum=0)
print fvasol.data_frame

pathway = escher.Builder(map_name='escher_map_RECON2_AA_metabolism', reaction_scale=escher_colors, 
                         reaction_data={reaction: flux for reaction, flux in fbasol.fluxes.items() }, reaction_styles=['color', 'size', 'text','abs'])
pathway.save_html('./predictions/PKU_noTyrosine_nor0399_noPhethptox2_34hpp.html',overwrite=True)
pathway.display_in_notebook()

### Discuss 
Even though this may be a simple example what does it teach us about the potential utility of the human metabolic map? 

Can you envision a future in which each individual has his or her genetic mutations mapped onto such a model?
Could we use similar tricks like discussed above to find ways (with food and drugs) around the blockages such mutations generatate?